In [2]:
# Dependencies
import pandas as pd
import requests
import json
import datetime
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy_utils import database_exists, create_database

from ps_wd import pswd

In [3]:
#Read the csv file 
sf_csv = "Registered_Business_Locations_-_San_Francisco.csv"
sf_df = pd.read_csv(sf_csv)
sf_df.head()

C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,Supervisor District,Neighborhoods - Analysis Boundaries,Business Corridor,Business Location,UniqueID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Neighborhoods
0,0031446-01-001,31446,Yong Kee Co Inc,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,...,3.0,Chinatown,NaN,POINT (-122.40721 37.795982),0031446-01-001-0031446-5300-5399-05-31-2013,104.0,6.0,3.0,6.0,104.0
1,0051039-02-001,51039,Busuttel Joseph F,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,...,8.0,Mission,NaN,POINT (-122.423996 37.75992),0051039-02-001-0051039-5300-5399-04-22-1977,37.0,3.0,5.0,20.0,37.0
2,0078067-01-999,78067,R. S. Hughes Company Inc,Hughes R S Co Inc,7303 Edgewater Dr Oakland Unit B,Oakland,CA,94621,10/01/1968,NaN,...,NaN,NaN,NaN,POINT (-122.20762 37.74483),0078067-01-999-0078067-4200-4299-10-01-1968,NaN,NaN,NaN,NaN,NaN
3,0080538-01-001,80538,Kwang Chun Tsang & So Lan,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,...,1.0,Outer Richmond,NaN,POINT (-122.491425 37.78295),0080538-01-001-0080538-5300-5399-07-20-1973,8.0,8.0,4.0,29.0,8.0
4,0135472-01-001,135472,Saddul Oscar Allado Md Inc,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,...,9.0,Mission,NaN,POINT (-122.419 37.757675),0135472-01-001-0135472-5400-5499-05-15-1982,53.0,3.0,2.0,20.0,53.0


In [4]:
# drop columns that you recognize as unnecessary 
sf_df.drop(["Location Id","Business Account Number","Ownership Name","Mail Address","Mail State", "Mail City","Mail Zipcode","Parking Tax","Transient Occupancy Tax","LIC Code","LIC Code Description","Supervisor District","Business Corridor","UniqueID","SF Find Neighborhoods","Current Police Districts","Current Supervisor Districts","Analysis Neighborhoods","Neighborhoods"], axis = 1, inplace = True) 

#drop columns that you recognize as unnecessary 
# sf_df.drop(["Location_Id","Business_Account_Number","Ownership_Name","Mail_Address","Mail_State", "Mail_City","Mail_Zipcode","Parking_Tax","Transient_Occupancy_Tax","LIC_Code","LIC_Code_Description","Supervisor_District","Business_Corridor","UniqueID","SF_Find_Neighborhoods","Current_Police_Districts","Current_Supervisor_Districts","Analysis_Neighborhoods","Neighborhoods"], axis = 1, inplace = True) 
  

sf_df.head()

,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,Location Start Date,Location End Date,NAICS Code,NAICS Code Description,Neighborhoods - Analysis Boundaries,Business Location
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982)
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992)
2,Hughes R S Co Inc,7303 Edgewater Dr Oakland Unit B,Oakland,CA,94621,10/01/1968,NaN,10/01/1968,03/30/2014,4200-4299,Wholesale Trade,NaN,POINT (-122.20762 37.74483)
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295)
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675)


In [6]:
#Get details of the data that is not based in San Francisco 
indexNames = sf_df[ sf_df['City'] != ("San Francisco" or "San+Francisco") ].index
# # Delete these row indexes from dataFrame
sf_df.drop(indexNames , inplace=True)
sf_df.head()

,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,Location Start Date,Location End Date,NAICS Code,NAICS Code Description,Neighborhoods - Analysis Boundaries,Business Location
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982)
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992)
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295)
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675)
6,Bakesale Betty/hot Dog Bill East 600,1 Warriors Way Lvl 600 East,San Francisco,CA,94158,07/01/1987,NaN,05/28/2019,NaN,7220-7229,Food Services,Mission Bay,POINT (-122.386406 37.768967)


In [7]:
#Create database
engine = create_engine('postgresql://postgres:'+ pswd + '@localhost:5432/original_db')
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url))

True


In [8]:
#Establish connection with the database
connection = engine.connect()

In [10]:
#Load the data to SQL database (a table named 'dow_jones' is created in postgresql)
sf_df.to_sql(name='original_table', con=engine, if_exists='replace', index=False)

In [11]:
# extract the DataFrame to an output csv file 
sf_df.to_csv('original_data.csv', encoding='utf-8', index=False)